<a href="https://colab.research.google.com/github/TaruSora/NLP_100knock/blob/main/100knock_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

90.機械翻訳のデータセットをダウンロードせよ．訓練データ，開発データ，評価データを整形し，必要に応じてトークン化などの前処理を行うこと．ただし，この段階ではトークンの単位として形態素（日本語）および単語（英語）を採用せよ.

In [1]:
# 90 データの準備

! wget http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
! tar xvzf kftt-data-1.0.tar.gz

! pip install -U --quiet ja_ginza
! python -m spacy download en_core_web_sm

! mkdir -p /content/ginza-data
! cat kftt-data-1.0/data/orig/kyoto-train.ja | sed 's/\s+/ /g' | ginzame > /content/ginza-data/train.ginza.ja
! cat kftt-data-1.0/data/orig/kyoto-dev.ja | sed 's/\s+/ /g' | ginzame > /content/ginza-data/dev.ginza.ja
! cat kftt-data-1.0/data/orig/kyoto-test.ja | sed 's/\s+/ /g' | ginzame > /content/ginza-data/test.ginza.ja

--2022-04-01 06:28:39--  http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
Resolving www.phontron.com (www.phontron.com)... 208.113.196.149
Connecting to www.phontron.com (www.phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99246893 (95M) [application/gzip]
Saving to: ‘kftt-data-1.0.tar.gz’

kftt-data-1.0.tar.g 100%[===================>]  94.65M  99.3MB/s    in 1.0s    

2022-04-01 06:28:42 (99.3 MB/s) - ‘kftt-data-1.0.tar.gz’ saved [99246893/99246893]

kftt-data-1.0/
kftt-data-1.0/data/
kftt-data-1.0/data/orig/
kftt-data-1.0/data/orig/kyoto-tune.en
kftt-data-1.0/data/orig/kyoto-dev.ja
kftt-data-1.0/data/orig/kyoto-dev.en
kftt-data-1.0/data/orig/kyoto-train.en
kftt-data-1.0/data/orig/kyoto-tune.ja
kftt-data-1.0/data/orig/kyoto-train.ja
kftt-data-1.0/data/orig/kyoto-test.ja
kftt-data-1.0/data/orig/kyoto-test.en
kftt-data-1.0/data/tok/
kftt-data-1.0/data/tok/kyoto-tune.en
kftt-data-1.0/data/tok/kyoto-dev.ja
kftt-data-1.0/dat

In [2]:
!mkdir -p /content/data
for src, dst in [
    ("/content/ginza-data/train.ginza.ja", "/content/data/train.ja"),
    ("/content/ginza-data/dev.ginza.ja", "/content/data/dev.ja"),
    ("/content/ginza-data/test.ginza.ja", "/content/data/test.ja"),
]:
    with open(src) as f:
        list = []
        tmp = []
        for x in f:
            x = x.strip()
            if x == 'EOS':
                list.append(' '.join(tmp))
                tmp = []
            elif x != '':
                tmp.append(x.split('\t')[0])
    with open(dst, 'w') as f:
        for line in list:
            print(line, file=f)

In [3]:
import re
import spacy

nlp = spacy.load("en_core_web_sm")
for src, dst in [
    ("kftt-data-1.0/data/orig/kyoto-train.en", "/content/data/train.en"),
    ("kftt-data-1.0/data/orig/kyoto-dev.en", "/content/data/dev.en"),
    ("kftt-data-1.0/data/orig/kyoto-test.en", "/content/data/test.en"),
]:
    with open(src) as f, open(dst, 'w') as g:
        for x in f:
            x = x.strip()
            x = re.sub(r'\s+', ' ', x)
            x = nlp.make_doc(x)
            x = ' '.join([doc.text for doc in x])
            print(x, file=g)

91.90で準備したデータを用いて，ニューラル機械翻訳のモデルを学習せよ（ニューラルネットワークのモデルはTransformerやLSTMなど適当に選んでよい）．

In [6]:
# 91 機械翻訳モデルの訓練

!git clone https://github.com/pytorch/fairseq
%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

!fairseq-preprocess -s ja -t en \
    --trainpref /content/data/train \
    --validpref /content/data/dev \
    --destdir /content/data \
    --thresholdsrc 5 \
    --thresholdtgt 5 \
    --workers 20

Cloning into 'fairseq'...
remote: Enumerating objects: 31117, done.
remote: Counting objects: 100% (748/748), done.
remote: Compressing objects: 100% (401/401), done.
remote: Total 31117 (delta 371), reused 618 (delta 335), pack-reused 30369
Receiving objects: 100% (31117/31117), 21.33 MiB | 27.03 MiB/s, done.
Resolving deltas: 100% (23070/23070), done.
/content/fairseq
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 123 kB 27.2 MB/s 
     |████████████████████████████████| 90 kB 9.0 MB/s 
     |████████████████████████████████| 223 kB 69.4 MB/s 
     |████████████████████████████████| 112 kB 55.6 MB/s 
     |████████████████████████████████| 596 kB 64.6 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=06e428efb7be983d395e

In [ ]:
!fairseq-train /content/data --arch transformer \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --lr 1e-5 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 3000 \
    --optimizer adam \
    --max-tokens 4000 \
    --max-epoch 3 \
    --dropout 0.1 \
    --clip-norm 0.0 \
    --weight-decay 0.0001 \
    --patience 5 \
    --no-epoch-checkpoints > 91.log \

!cp /content/checkpoints/checkpoint_best.pt "/content/drive/MyDrive/Colab Notebooks/checkpoint_best.pt"

epoch 001:   0% 0/3618 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
epoch 001:   8% 287/3618 [2:16:56<24:34:47, 26.56s/it, loss=15.368, nll_loss=15.245, ppl=38834.4, wps=120.2, ups=0.03, wpb=3459.6, bsz=118.2, num_updates=200, lr=6.66667e-07, gnorm=7.958, train_wall=2872, wall=5788]

92.91で学習したニューラル機械翻訳モデルを用い，与えられた（任意の）日本語の文を英語に翻訳するプログラムを実装せよ．

In [ ]:
# 92 機械翻訳モデルの適用

!fairseq-interactive --path /content/checkpoints/checkpoint_best.pt /content/data < /content/data/test.ja | grep '^H' | cut -f3 > translate_92.out

93.91で学習したニューラル機械翻訳モデルの品質を調べるため，評価データにおけるBLEUスコアを測定せよ．

In [ ]:
# 93 BLUEスコアの計測

!fairseq-score --sys translate_92.out --ref /content/data/test.en

94.91で学習したニューラル機械翻訳モデルで翻訳文をデコードする際に，ビーム探索を導入せよ．ビーム幅を1から100くらいまで適当に変化させながら，開発セット上のBLEUスコアの変化をプロットせよ．

In [ ]:
# 94 ビーム探索

!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best.pt" < /content/data/test.ja --beam 1 | grep '^H' | cut -f3 > translate_92_1.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best.pt" < /content/data/test.ja --beam 25 | grep '^H' | cut -f3 > translate_92_25.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best.pt" < /content/data/test.ja --beam 50 | grep '^H' | cut -f3 > translate_92_50.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best.pt" < /content/data/test.ja --beam 75 | grep '^H' | cut -f3 > translate_92_75.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best.pt" < /content/data/test.ja --beam 100 | grep '^H' | cut -f3 > translate_92_100.out

In [ ]:
!fairseq-score --sys translate_92_1.out --ref /content/data/test.en > /content/BLEU_score_1
!fairseq-score --sys translate_92_25.out --ref /content/data/test.en > /content/BLEU_score_25
!fairseq-score --sys translate_92_50.out --ref /content/data/test.en > /content/BLEU_score_50
!fairseq-score --sys translate_92_75.out --ref /content/data/test.en > /content/BLEU_score_75
!fairseq-score --sys translate_92_100.out --ref /content/data/test.en > /content/BLEU_score_100

In [ ]:
import matplotlib.pyplot as plt
import re

def cal_score(file):
    with open(file) as f:
        line = f.readlines()[1]
        line = re.search(r"(?<=BLEU4 = )\d*\.\d*(?=,)", line)
        return float(line.group())

x = [1, 25, 50, 75, 100]
y = [cal_score("/content/BLEU_score_{}".format(n)) for n in x]
plt.plot(x, y)
plt.xlabel("beam_size")
plt.ylabel("BLEU")
plt.show()

95.トークンの単位を単語や形態素からサブワードに変更し，91-94の実験を再度実施せよ．

In [ ]:
!pip install sentencepiece
!pip install subword-nmt

In [ ]:
# 95 サブワード化

import sentencepiece as spm
import re

spm.SentencePieceTrainer.Train("--input=kftt-data-1.0/data/orig/kyoto-train.ja --model_prefix=kyoto_ja --vocab_size=16000 --character_coverage=1.0")

sp = spm.SentencePieceProcessor()
sp.Load("kyoto_ja.model")

for src, dst in [
    ("kftt-data-1.0/data/orig/kyoto-train.ja", "/content/data/train.ja"),
    ("kftt-data-1.0/data/orig/kyoto-dev.ja", "/content/data/dev.ja"),
    ("kftt-data-1.0/data/orig/kyoto-test.ja", "/content/data/test.ja"),
]:
    with open(src) as f, open(dst, 'w') as g:
        for x in f:
            x = x.strip()
            x = re.sub(r'\s+', ' ', x)
            x = sp.encode_as_pieces(x)
            x = " ".join(x)
            print(x, file=g)

In [ ]:
!subword-nmt learn-bpe -s 16000 < kftt-data-1.0/data/orig/kyoto-train.en > kyoto_en.codes
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/orig/kyoto-train.en > /content/data/train.en
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/orig/kyoto-dev.en > /content/data/dev.en
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/orig/kyoto-test.en > /content/data/test.en

In [ ]:
# 前処理

!git clone https://github.com/pytorch/fairseq
%cd /content/fairseq/
!python -m pip install --editable .
%cd /content
os.environ['PYTHONPATH'] += ":/content/fairseq/"

!fairseq-preprocess -s ja -t en \
    --trainpref /content/data/train \
    --validpref /content/data/dev \
    --destdir /content/data \
    --thresholdsrc 5 \
    --thresholdtgt 5 \
    --workers 20

In [ ]:
# 訓練

!fairseq-train /content/data --arch transformer \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --lr 1e-5 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 3000 \
    --optimizer adam \
    --max-tokens 4000 \
    --max-epoch 3 \
    --dropout 0.1 \
    --clip-norm 0.0 \
    --weight-decay 0.0001 \
    --patience 5 \
    --no-epoch-checkpoints > 95.log \

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

! cp /content/checkpoints/checkpoint_best.pt "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt"

In [ ]:
# モデルの適用
!fairseq-interactive --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt" /content/data < /content/data/test.ja | grep '^H' | cut -f3 > translate_95.out

In [ ]:
# BLUEスコアの計測
!fairseq-score --sys translate_95.out --ref /content/data/test.en

In [ ]:
# ビーム探索
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt" < /content/data/test.ja --beam 1 | grep '^H' | cut -f3 > translate_95_1.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt" < /content/data/test.ja --beam 25 | grep '^H' | cut -f3 > translate_95_25.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt" < /content/data/test.ja --beam 50 | grep '^H' | cut -f3 > translate_95_50.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt" < /content/data/test.ja --beam 75 | grep '^H' | cut -f3 > translate_95_75.out
!fairseq-interactive /content/data --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_2.pt" < /content/data/test.ja --beam 100 | grep '^H' | cut -f3 > translate_95_100.out

!fairseq-score --sys translate_95_1.out --ref /content/data/test.en > /content/BLEU_score_1
!fairseq-score --sys translate_95_25.out --ref /content/data/test.en > /content/BLEU_score_25
!fairseq-score --sys translate_95_50.out --ref /content/data/test.en > /content/BLEU_score_50
!fairseq-score --sys translate_95_75.out --ref /content/data/test.en > /content/BLEU_score_75
!fairseq-score --sys translate_95_100.out --ref /content/data/test.en > /content/BLEU_score_100

In [ ]:
import matplotlib.pyplot as plt
import re

def cal_score(file):
    with open(file) as f:
        line = f.readlines()[1]
        line = re.search(r"(?<=BLEU4 = )\d*\.\d*(?=,)", line)
        return float(line.group())

x = [1, 25, 50, 75, 100]
y = [cal_score("/content/BLEU_score_{}".format(n)) for n in x]
plt.plot(x, y)
plt.xlabel("beam_size")
plt.ylabel("BLEU")
plt.show()

96.Tensorboardなどのツールを用い，ニューラル機械翻訳モデルが学習されていく過程を可視化せよ．可視化する項目としては，学習データにおける損失関数の値とBLEUスコア，開発データにおける損失関数の値とBLEUスコアなどを採用せよ．

In [ ]:
!pip install tensorboardX

In [ ]:
# 96 学習過程の可視化

!fairseq-train /content/data --arch transformer\
    --tensorboard-logdir log96 \
    --save-dir save96 \
    --max-epoch 3 \
    --optimizer adam --clip-norm 1.0 \
    --lr 1e-3 --lr-scheduler inverse_sqrt --warmup-updates 3000 \
    --dropout 0.1 --weight-decay 0.0001 \
    --update-freq 1 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4000 > 96.log

In [ ]:
%load_ext tensorboard
%tensorboard --logdir log96

97.ニューラルネットワークのモデルや，そのハイパーパラメータを変更しつつ，開発データにおけるBLEUスコアが最大となるモデルとハイパーパラメータを求めよ.

In [ ]:
# 97 ハイパー・パラメータの調整

!fairseq-train /content/data --arch transformer \
    --save-dir save97 \
    --max-epoch 3 \
    --optimizer adam --clip-norm 1.0 \
    --lr 3e-3 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --update-freq 1 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 6000 > 97.log

!cp /content/save97/checkpoint_best.pt "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_3.pt"

!fairseq-interactive --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_3.pt" /content/data < /content/data/test.ja --beam 10 | grep '^H' | cut -f3 > translate_97.out

!fairseq-score --sys translate_97.out --ref /content/data/test.en

98.Japanese-English Subtitle Corpus (JESC)やJParaCrawlなどの翻訳データを活用し，KFTTのテストデータの性能向上を試みよ．

In [ ]:
# 98 ドメイン適応

!fairseq-preprocess -s ja -t en \
    --trainpref "/content/drive/MyDrive/Colab Notebooks/train.jesc" \
    --validpref /content/data/dev \
    --destdir /content/data \
    --thresholdsrc 5 \
    --thresholdtgt 5 \
    --workers 20

!fairseq-train /content/data --arch transformer \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --lr 1e-5 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 3000 \
    --optimizer adam \
    --max-tokens 4000 \
    --max-epoch 1 \
    --dropout 0.1 \
    --clip-norm 0.0 \
    --weight-decay 0.0001 \
    --patience 5 \
    --no-epoch-checkpoints > 98_1.log \

! cp /content/checkpoints/checkpoint_best.pt "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_5.pt"

!fairseq-interactive --path "/content/drive/MyDrive/Colab Notebooks/checkpoint_best_5.pt" /content/data < /content/data/test.ja | grep '^H' | cut -f3 > translate_98_2.out

!fairseq-score --sys translate_98_2.out --ref /content/data/test.en

99.ユーザが翻訳したい文を入力すると，その翻訳結果がウェブブラウザ上で表示されるデモシステムを構築せよ.

In [ ]:
!pip install flask-ngrok
!pip install fairseq

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
from fairseq.models.transformer import TransformerModel

model = TransformerModel.from_pretrained(
    '/content/checkpoints/',
    checkpoint_file="checkpoint_best.pt",
    data_name_or_path="/content/data",
)

app = Flask(__name__,template_folder="(前略)/templates")
run_with_ngrok(app)

@app.route("/", methods=["GET"])
def get():
    return render_template("index.html", \
        title = "翻訳", \
        message = "入力してください")

@app.route("/", methods=["POST"])
def post():
    text = request.form["name"]
    return render_template("index.html", \
        title = "翻訳", \
        output = model.translate(text, beam=1)
        message = text+"　　　　　　　→　"+output)

app.run()